In [1]:
# encoding=utf8  
import numpy as np
import tensorflow as tf

def unpickle(file):
  import _pickle as cPickle
  fo = open(file, 'rb')
  dict = cPickle.load(fo,encoding='latin1')
  fo.close()
  if 'data' in dict:
    dict['data'] = dict['data'].reshape((-1, 3, 32, 32)).swapaxes(1, 3).swapaxes(1, 2).reshape(-1, 32*32*3) / 256.

  return dict

In [2]:
def load_data_one(f):
  batch = unpickle(f)
  data = batch['data']
  labels = batch['labels']
  print ("Loading %s: %d" % (f, len(data)))
  return data, labels


In [3]:

def load_data(files, data_dir, label_count):
  data, labels = load_data_one(data_dir + '/' + files[0])
  for f in files[1:]:
    data_n, labels_n = load_data_one(data_dir + '/' + f)
    data = np.append(data, data_n, axis=0)
    labels = np.append(labels, labels_n, axis=0)
  labels = np.array([ [ float(i == label) for i in range(label_count) ] for label in labels ])
  return data, labels

In [4]:
def run_in_batch_avg(session, tensors, batch_placeholders, feed_dict={}, batch_size=200):                              
  res = [ 0 ] * len(tensors)                                                                                           
  batch_tensors = [ (placeholder, feed_dict[ placeholder ]) for placeholder in batch_placeholders ]                    
  total_size = len(batch_tensors[0][1])                                                                                
  batch_count = (total_size + batch_size - 1) / batch_size                                                             
  for batch_idx in range(batch_count):                                                                                
    current_batch_size = None                                                                                          
    for (placeholder, tensor) in batch_tensors:                                                                        
      batch_tensor = tensor[ batch_idx*batch_size : (batch_idx+1)*batch_size ]                                         
      current_batch_size = len(batch_tensor)                                                                           
      feed_dict[placeholder] = tensor[ batch_idx*batch_size : (batch_idx+1)*batch_size ]                               
    tmp = session.run(tensors, feed_dict=feed_dict)                                                                    
    res = [ r + t * current_batch_size for (r, t) in zip(res, tmp) ]                                                   
  return [ r / float(total_size) for r in res ]


In [5]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.01)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.01, shape=shape)
  return tf.Variable(initial)

def conv2d(input, in_features, out_features, kernel_size, with_bias=False):
  W = weight_variable([ kernel_size, kernel_size, in_features, out_features ])
  conv = tf.nn.conv2d(input, W, [ 1, 1, 1, 1 ], padding='SAME')
  if with_bias:
    return conv + bias_variable([ out_features ])
  return conv

def batch_activ_conv(current, in_features, out_features, kernel_size, is_training, keep_prob):
  current = tf.contrib.layers.batch_norm(current, scale=True, is_training=is_training, updates_collections=None)
  current = tf.nn.relu(current)
  current = conv2d(current, in_features, out_features, kernel_size)
  current = tf.nn.dropout(current, keep_prob)
  return current

def block(input, layers, in_features, growth, is_training, keep_prob):
  current = input
  features = in_features
  for idx in range(layers):
    tmp = batch_activ_conv(current, features, growth, 3, is_training, keep_prob)
    current = tf.concat((current, tmp),3)
    features += growth
  return current, features

def avg_pool(input, s):
  return tf.nn.avg_pool(input, [ 1, s, s, 1 ], [1, s, s, 1 ], 'VALID')



In [6]:
data_dir = './data'
image_size = 32
image_dim = image_size * image_size * 3
# meta = unpickle(data_dir + '/batches.meta')
# label_names = meta['label_names']
# label_count = len(label_names)
label_count = 10
# train_files = [ 'data_batch_%d' % d for d in range(1, 6) ]
# train_data, train_labels = load_data(train_files, data_dir, label_count)
# pi = np.random.permutation(len(train_data))
# train_data, train_labels = train_data[pi], train_labels[pi]
# test_data, test_labels = load_data([ 'test_batch' ], data_dir, label_count)
# print ("Train:", np.shape(train_data), np.shape(train_labels))
# print ("Test:", np.shape(test_data), np.shape(test_labels))
# data = { 'train_data': train_data,
#   'train_labels': train_labels,
#   'test_data': test_data,
#   'test_labels': test_labels }
depth = 40

In [7]:

weight_decay = 1e-4
layers = int((depth - 4) / 3)
graph = tf.Graph()

xs = tf.placeholder("float", shape=[None, image_dim])
ys = tf.placeholder("float", shape=[None, label_count])
lr = tf.placeholder("float", shape=[])
keep_prob = tf.placeholder(tf.float32)
is_training = tf.placeholder("bool", shape=[])


current = tf.reshape(xs, [ -1, 32, 32, 3 ])
current = conv2d(current, 3, 16, 3)

current, features = block(current, layers, 16, 12, is_training, keep_prob)
current = batch_activ_conv(current, features, features, 1, is_training, keep_prob)
current = avg_pool(current, 2)
current, features = block(current, layers, features, 12, is_training, keep_prob)
current = batch_activ_conv(current, features, features, 1, is_training, keep_prob)
current = avg_pool(current, 2)
current, features = block(current, layers, features, 12, is_training, keep_prob)

current = tf.contrib.layers.batch_norm(current, scale=True, is_training=is_training, updates_collections=None)
current = tf.nn.relu(current)
current = avg_pool(current, 8)
final_dim = features
current = tf.reshape(current, [ -1, final_dim ])
Wfc = weight_variable([ final_dim, label_count ])
bfc = bias_variable([ label_count ])
ys_ = tf.nn.softmax( tf.matmul(current, Wfc) + bfc )

cross_entropy = -tf.reduce_mean(ys * tf.log(ys_ + 1e-12))
l2 = tf.add_n([tf.nn.l2_loss(var) for var in tf.trainable_variables()])
train_step = tf.train.MomentumOptimizer(lr, 0.9, use_nesterov=True).minimize(cross_entropy + l2 * weight_decay)
correct_prediction = tf.equal(tf.argmax(ys_, 1), tf.argmax(ys, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    


In [8]:
para_dict={}
for k in tf.global_variables():
    if k not in tf.contrib.framework.get_variables_by_suffix('Momentum'):
            para_dict[k.name[:-2]] = k

In [9]:
sess=tf.InteractiveSession()
saver = tf.train.Saver(para_dict)
saver.restore(sess,'./prunemodel/stage2/inc100adj/prune100ar_92969_10ok.ckpt')

INFO:tensorflow:Restoring parameters from ./prunemodel/stage2/inc100adj/prune100ar_92969_10ok.ckpt


In [10]:
import config


In [11]:
import cv2
# Define criteria = ( type, max_iter = 10 , epsilon = 1.0 )
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

# Set flags (Just to avoid line break in the code)
flags = cv2.KMEANS_RANDOM_CENTERS
def weight_shared(weight_arr, name="None", nmeans = 4, **kwargs):
    """Apply weight shared """
    
    weight_arr_tem = np.reshape(weight_arr,[-1])
    weight_arr_nzidx = np.nonzero(weight_arr_tem)

    weight_arr_nztem = weight_arr_tem[weight_arr_nzidx]
    w_cenidx = np.ones(np.shape(weight_arr_tem)) * (nmeans)

#     weight_arr_nzidx = np.nonzero(weight_arr_tem)
#     weight_arr_nz = weight_arr_tem[weight_arr_nzidx]
    _,labels,centroids = cv2.kmeans(np.float32(np.reshape(weight_arr_nztem,[-1,1])),nmeans,None,criteria,10,flags)
    #kmeans.fit(np.float32(np.reshape(weight_arr_tem,[-1,1])))
    #w_cen=kmeans.cluster_centers_.tolist()
    w_cen = centroids
#     w_cen.append([0])
    w_cen = np.asarray(w_cen)
#     w_cenidx=np.ones(np.shape(weight_arr_tem)) * (nmeans)
#     print(np.shape(weight_arr_nzidx))
#     print(np.shape(w_cenidx))
#     print(np.shape(w_cenidx[weight_arr_nzidx]))
#     print(np.shape(labels))

    w_cenidx[weight_arr_nzidx] = np.reshape(labels,[-1])
#     w_cenidx = np.reshape(w_cenidx,[-1])
#     print(np.shape(w_cenidx))
    #w_cenidx=kmeans.labels_.tolist()
    # w_nzcenidx = kmeans.labels_.tolist()
    # w_cenidx[weight_arr_nzidx] = w_nzcenidx
    w_cenidx = np.reshape(w_cenidx,np.shape(weight_arr))
    return weight_arr, w_cenidx, w_cen

In [12]:
def apply_prune_weightshared(weights, prune_dict):

    dict_cenidx = {}
    dict_cen = {}
    for target in config.exc_para:
        wl = target

        num_cen = config.kmeans_para[wl] 

  

        # Get target layer's weights
        weight_obj = weights[wl]
        weight_arr = weight_obj.eval()

        # Apply pruning
        weight_arr, w_cenidx, w_cen = weight_shared(weight_arr, name=wl,
                                            nmeans=num_cen)

        # Store pruned weights as tensorflow objects
        dict_cenidx[wl] = w_cenidx
        dict_cen[wl] = w_cen
        weight_arr_tem = np.reshape(weight_arr,[-1])
        w_cenidx_tem = np.reshape(w_cenidx,[-1])
        w_cenidx_tem = w_cenidx_tem.astype(int)
        w_cen = w_cen

        for i in range(len(weight_arr_tem)):
            if w_cenidx_tem[i] == num_cen:
                weight_arr_tem[i] = 0
            else:
                weight_arr_tem[i]=w_cen[w_cenidx_tem[i]]
        weight_arr = np.reshape(weight_arr_tem,np.shape(weight_arr))
        #print(dict_cen[wl])
        sess.run(weight_obj.assign(weight_arr))
    return dict_cenidx,dict_cen

In [13]:
prune_dict = {}
for target in config.exc_para:
    wl =target
    weight_obj = para_dict[wl]
    prune_dict[wl] = np.zeros_like(weight_obj.eval())

In [14]:
dict_cenidx,dict_cen = apply_prune_weightshared(para_dict, prune_dict)





In [15]:
saver.save(sess,'./kmeansmodel/stage1/100pkmeans64.ckpt')

'./kmeansmodel/stage1/100pkmeans64.ckpt'

In [16]:
import pickle
# create dict
# save dict
f1 = open("C:/Users/lhlne/Desktop/project/densenet/kmeansmodel/stage1/100pkmeans64.txt","wb")
pickle.dump(dict_cenidx, f1)
f1.close()
# load dict


In [17]:
import pickle
f2 = open("C:/Users/lhlne/Desktop/project/densenet/kmeansmodel/stage1/100pkmeans64.txt","rb")
load_list = pickle.load(f2)
f2.close()
# print 
print(load_list)

{'Variable_2': array([[[[ 64.,  64.,  64., ...,  64.,  64.,  64.],
         [ 64.,  64.,  64., ...,  64.,  64.,  64.],
         [ 64.,  64.,  64., ...,  64.,  64.,  64.],
         ..., 
         [ 64.,  64.,  64., ...,   6.,  64.,  64.],
         [ 64.,  64.,  64., ...,  64.,  64.,  64.],
         [ 64.,  64.,  64., ...,  64.,  64.,  64.]],

        [[ 64.,  64.,  64., ...,  64.,  64.,  64.],
         [ 64.,  64.,  64., ...,  64.,  64.,  64.],
         [ 64.,  64.,  64., ...,  64.,  64.,  64.],
         ..., 
         [ 64.,  64.,  64., ...,   6.,  64.,  63.],
         [ 64.,  64.,  64., ...,  64.,  64.,  64.],
         [ 64.,  64.,  64., ...,  64.,  64.,  64.]],

        [[ 64.,  64.,  64., ...,  64.,  64.,  64.],
         [ 64.,  64.,  64., ...,  64.,  64.,  64.],
         [ 64.,  64.,  64., ...,  64.,  64.,  64.],
         ..., 
         [ 64.,  64.,  64., ...,  64.,  64.,   6.],
         [ 64.,  64.,  64., ...,  64.,  64.,  64.],
         [ 64.,  64.,  64., ...,  64.,  64.,  64.]]]

In [19]:
para_dict['Variable_39'].eval()

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [-0.46109787, -0.22247255,  0.77080566, ...,  0.84448045,
        -0.25070801, -0.3224116 ],
       [-0.25070801, -0.2319456 , -0.34873989, ..., -0.30235809,
        -0.25070801, -0.17267427],
       [ 0.84448045,  0.84448045, -0.08770052, ..., -0.17267427,
        -0.25070801, -0.26271957]], dtype=float32)

In [15]:
dict_cen

NameError: name 'dict_cen' is not defined

In [16]:
from matplotlib import pyplot as plt
for i in para_dict.items():
    if 'Variable' in i[0]:
        a = para_dict[i[0]].eval()
        b = np.reshape(a,[-1])
        c = sum(b ==0)
        d = np.shape(b)
   #     e = np.percentile(b, 75)
        print(c)
        print(d)
 #       print(e)
   #     print(int(np.log10(abs(e))))
        print('percentage',c/d)
#         print(sum(b ==0))
#         plt.hist(abs(b)) 
#         plt.title(i[0]) 
#         plt.show()

0
(432,)
percentage [ 0.]
518
(1728,)
percentage [ 0.29976852]
1512
(3024,)
percentage [ 0.5]
2160
(4320,)
percentage [ 0.5]
2808
(5616,)
percentage [ 0.5]
3456
(6912,)
percentage [ 0.5]
4104
(8208,)
percentage [ 0.5]
4752
(9504,)
percentage [ 0.5]
5400
(10800,)
percentage [ 0.5]
6048
(12096,)
percentage [ 0.5]
6696
(13392,)
percentage [ 0.5]
7344
(14688,)
percentage [ 0.5]
7992
(15984,)
percentage [ 0.5]
6400
(25600,)
percentage [ 0.25]
8640
(17280,)
percentage [ 0.5]
9288
(18576,)
percentage [ 0.5]
9936
(19872,)
percentage [ 0.5]
10584
(21168,)
percentage [ 0.5]
11232
(22464,)
percentage [ 0.5]
11880
(23760,)
percentage [ 0.5]
12528
(25056,)
percentage [ 0.5]
13176
(26352,)
percentage [ 0.5]
13824
(27648,)
percentage [ 0.5]
14472
(28944,)
percentage [ 0.5]
15120
(30240,)
percentage [ 0.5]
15768
(31536,)
percentage [ 0.5]
27724
(92416,)
percentage [ 0.29999134]
16416
(32832,)
percentage [ 0.5]
17064
(34128,)
percentage [ 0.5]
17712
(35424,)
percentage [ 0.5]
18360
(36720,)
percentage 

In [45]:
a = parm_dict['Variable_39'].eval()
b = np.reshape(a,[-1])
print(sum(b <=0.00001))
print(np.shape(b))
print('percentage',a/b)

(4480,)

In [27]:
parm_dict={}
for k in tf.trainable_variables():
    parm_dict[k.name] = k
with tf.Session(graph=graph) as session:
    saver= tf.train.Saver(parm_dict) 
    saver.restore(session,"weights_densenet")

Variable:0
BatchNorm/beta:0
BatchNorm/gamma:0
Variable_1:0
BatchNorm_1/beta:0
BatchNorm_1/gamma:0
Variable_2:0
BatchNorm_2/beta:0
BatchNorm_2/gamma:0
Variable_3:0
BatchNorm_3/beta:0
BatchNorm_3/gamma:0
Variable_4:0
BatchNorm_4/beta:0
BatchNorm_4/gamma:0
Variable_5:0
BatchNorm_5/beta:0
BatchNorm_5/gamma:0
Variable_6:0
BatchNorm_6/beta:0
BatchNorm_6/gamma:0
Variable_7:0
BatchNorm_7/beta:0
BatchNorm_7/gamma:0
Variable_8:0
BatchNorm_8/beta:0
BatchNorm_8/gamma:0
Variable_9:0
BatchNorm_9/beta:0
BatchNorm_9/gamma:0
Variable_10:0
BatchNorm_10/beta:0
BatchNorm_10/gamma:0
Variable_11:0
BatchNorm_11/beta:0
BatchNorm_11/gamma:0
Variable_12:0
BatchNorm_12/beta:0
BatchNorm_12/gamma:0
Variable_13:0
BatchNorm_13/beta:0
BatchNorm_13/gamma:0
Variable_14:0
BatchNorm_14/beta:0
BatchNorm_14/gamma:0
Variable_15:0
BatchNorm_15/beta:0
BatchNorm_15/gamma:0
Variable_16:0
BatchNorm_16/beta:0
BatchNorm_16/gamma:0
Variable_17:0
BatchNorm_17/beta:0
BatchNorm_17/gamma:0
Variable_18:0
BatchNorm_18/beta:0
BatchNorm_18

In [7]:
depth = 40
weight_decay = 1e-4
layers = int((depth - 4) / 3)

xs = tf.placeholder("float", shape=[None, image_dim])
ys = tf.placeholder("float", shape=[None, label_count])
lr = tf.placeholder("float", shape=[])
keep_prob = tf.placeholder(tf.float32)
is_training = tf.placeholder("bool", shape=[])


current = tf.reshape(xs, [ -1, 32, 32, 3 ])
current = conv2d(current, 3, 16, 3)

current, features = block(current, layers, 16, 12, is_training, keep_prob)
current = batch_activ_conv(current, features, features, 1, is_training, keep_prob)
current = avg_pool(current, 2)
current, features = block(current, layers, features, 12, is_training, keep_prob)
current = batch_activ_conv(current, features, features, 1, is_training, keep_prob)
current = avg_pool(current, 2)
current, features = block(current, layers, features, 12, is_training, keep_prob)

current = tf.contrib.layers.batch_norm(current, scale=True, is_training=is_training, updates_collections=None)
current = tf.nn.relu(current)
current = avg_pool(current, 8)
final_dim = features
current = tf.reshape(current, [ -1, final_dim ])
Wfc = weight_variable([ final_dim, label_count ])
bfc = bias_variable([ label_count ])
ys_ = tf.nn.softmax( tf.matmul(current, Wfc) + bfc )

cross_entropy = -tf.reduce_mean(ys * tf.log(ys_ + 1e-12))
l2 = tf.add_n([tf.nn.l2_loss(var) for var in tf.trainable_variables()])
train_step = tf.train.MomentumOptimizer(lr, 0.9, use_nesterov=True).minimize(cross_entropy + l2 * weight_decay)
correct_prediction = tf.equal(tf.argmax(ys_, 1), tf.argmax(ys, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

sess = tf.Session()
batch_size = 64
learning_rate = 0.1
tf.global_variables_initializer()
    # saver = tf.train.Saver()
    # train_data, train_labels = data['train_data'], data['train_labels']
    # batch_count = int(len(train_data) / batch_size)
    # batches_data = np.split(train_data[:batch_count * batch_size], batch_count)
    # batches_labels = np.split(train_labels[:batch_count * batch_size], batch_count)
    # print ("Batch per epoch: ", batch_count)
    # for epoch in range(1, 1+300):
    #   if epoch == 150: learning_rate = 0.01
    #   if epoch == 225: learning_rate = 0.001
    #   for batch_idx in range(batch_count):
    #     xs_, ys_ = batches_data[batch_idx], batches_labels[batch_idx]
    #     batch_res = session.run([ train_step, cross_entropy, accuracy ],
    #       feed_dict = { xs: xs_, ys: ys_, lr: learning_rate, is_training: True, keep_prob: 0.8 })
    #     if batch_idx % 100 == 0: print (epoch, batch_idx, batch_res[1:])

    #   save_path = saver.save(session, 'densenet_%d.ckpt' % epoch)
    #   test_results = run_in_batch_avg(session, [ cross_entropy, accuracy ], [ xs, ys ],
    #       feed_dict = { xs: data['test_data'], ys: data['test_labels'], is_training: False, keep_prob: 1. })
    #   print (epoch, batch_res[1:], test_results)


<tf.Operation 'init' type=NoOp>